In [1]:
from contextily import add_basemap
import geopandas as gpd
import geoplot as gplt
from shapely.wkt import loads

# 导入matplotlib.pyplot模块
import matplotlib.pyplot as plt
# 设置全局字体为time newroman，字体为粗体
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.weight"] = "bold"

# 导入必要的库
import numpy as np # 用于科学计算
import networkx as nx # 用于网络分析和可视化
import copy
import pickle
import pandas as pd

import networkx as nx
import random
import os
from PIL import Image
import imageio

def create_gif_from_images(directory, output_filename, duration):
    # 获取所有图片文件
    filenames = sorted([os.path.join(directory, fn) for fn in os.listdir(directory) if fn.lower().endswith('.png') or fn.lower().endswith('.jpg')])

    # 读取所有图片
    images = [Image.open(filename) for filename in filenames]

    # 将图片保存为GIF
    imageio.mimsave(output_filename, images, duration=duration)

if __name__ == "__main__":
    directory = 'images'
    output_filename = 'output_2.gif'
    duration = 2  # 每张图片显示的时间，单位是秒
    create_gif_from_images(directory, output_filename, duration)

In [ ]:
points = pd.read_csv('data/points.csv')
points['geometry'] = points['geometry'].apply(loads)
points = gpd.GeoDataFrame(points, geometry='geometry')
# dt_2019_mini[['geometry']].to_csv('data/points.csv')
shenzhen = gpd.read_file('data/深圳市.json')
# 将shenzhen转化成geodataframe
shenzhen = gpd.GeoDataFrame(shenzhen)
# 获得深圳的边界
shenzhen_boundary = shenzhen.total_bounds


with open('test_data_3/y_states', 'rb') as f:
    y_states = pickle.load(f)

with open('test_data_3/x_states', 'rb') as f:
    x_states = pickle.load(f)

In [ ]:
for d in range(len(x_states)):
    points['state'] = x_states[d]

    # 去掉death的点
    dt_new = points[points['state'] == 'remaining']
    # 设置图层大小
    ini_ax = gplt.webmap(shenzhen,
        provider='http://{s}.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}{r}.png',  figsize=(20, 10))

    # 绘制深圳区划
    ax = gplt.polyplot(df=shenzhen,
                    facecolor='black',
                    edgecolor='white',
                    alpha=0.65,
                    ax=ini_ax)

    # 绘制餐馆散点
    ax = gplt.pointplot(df=dt_new,
                        ax=ax,
                        extent=shenzhen.total_bounds,
                        color='#edc00d',
                        alpha=0.3,
                        linewidth=0.2,
                        s=4.5)

    # 添加标题
    ax.set_title(f'Shenzhen City Restaurants at Day {d}', fontsize=20, fontweight='bold')

    plt.savefig(f"images/image_{d}.png", bbox_inches='tight', pad_inches=0, dpi=300)
    print(f'完成第{d}天的绘图')

In [ ]:
image_files = os.listdir('images')

# 设置gif文件保存地址
gif_name = 'res_states_shenzhen.mp4'
# gif_name_opt = '/home/mw/temp/tom-opt.gif' #优化后生成的gif文件名称
# 读取图片文件
frames = [imageio.imread('images/'+f) for f in image_files]
# 将图片保存为GIF
imageio.mimsave(output_filename, frames, duration=duration)

### 构建用于可视化的耦合网络

In [ ]:
num = 10000

In [ ]:
with open('data/layerY_mobility', 'rb') as f:
    layer_y = pickle.load(f)

with open('data/layerX_restaurant', 'rb') as f:
    layer_x = pickle.load(f)

with open('data/layerZ_coupling_new', 'rb') as f:
    layer_z = pickle.load(f)

In [ ]:
layer_x_mini = layer_x.loc[np.random.choice(range(len(layer_x)), replace=False, size=num), :]
layer_y_mini = layer_y.loc[np.random.choice(range(len(layer_y)), replace=False, size=num), :]
layer_z_mini = layer_z.loc[np.random.choice(range(len(layer_z)), replace=False, size=num), :]

In [ ]:
layers_mini = pd.concat([layer_x_mini, layer_y_mini, layer_z_mini])
layers_mini.to_csv('layers_mini.txt', sep='\t', index=False, header=True)

In [ ]:
edge_mini = layers_mini.copy()
edge_mini = edge_mini.drop(labels='Weight', axis=1)
edge_mini['Color'] = "#3D3D3D"
edge_mini.to_csv('edge_attr.txt', sep='\t', index=False, header=True)

In [ ]:
x_color = '#648177'
y_color = '#FFC189'
size = 0.35

In [ ]:
x_node_attr_source = layer_x_mini[['SourceNode', 'SourceLayer']]
x_node_attr_source.columns = ['Node', 'Layer']
x_node_attr_source['Color'] = x_color
x_node_attr_source['Size'] = size 


x_node_attr_target = layer_x_mini[['TargetNode', 'TargetLayer']]
x_node_attr_target.columns = ['Node', 'Layer']
x_node_attr_target['Color'] = x_color
x_node_attr_target['Size'] = size 

y_node_attr_source = layer_y_mini[['SourceNode', 'SourceLayer']]
y_node_attr_source.columns = ['Node', 'Layer']
y_node_attr_source['Color'] = y_color
y_node_attr_source['Size'] = size 

y_node_attr_target = layer_y_mini[['TargetNode', 'TargetLayer']]
y_node_attr_target.columns = ['Node', 'Layer']
y_node_attr_target['Color'] = y_color
y_node_attr_target['Size'] = size 

z_node_attr_source = layer_z_mini[['SourceNode', 'SourceLayer']]
z_node_attr_source.columns = ['Node', 'Layer']
z_node_attr_source['Color'] = x_color
z_node_attr_source['Size'] = size 

z_node_attr_target = layer_z_mini[['TargetNode', 'TargetLayer']]
z_node_attr_target.columns = ['Node', 'Layer']
z_node_attr_target['Color'] = y_color
z_node_attr_target['Size'] = size 

node_attr = pd.concat([x_node_attr_source, x_node_attr_target, y_node_attr_source, y_node_attr_target, z_node_attr_source, z_node_attr_target])
# node_attr.to_csv('data/node_attr.txt', sep='\t', index=False, header=True)

C:\Users\wujlin\AppData\Local\Temp\ipykernel_7588\4202133210.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_node_attr_source['Color'] = x_color
C:\Users\wujlin\AppData\Local\Temp\ipykernel_7588\4202133210.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_node_attr_source['Size'] = size
C:\Users\wujlin\AppData\Local\Temp\ipykernel_7588\4202133210.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead